In [ ]:
!pip install -q transformers datasets torch evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import json
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AlbertTokenizer,
    AlbertForQuestionAnswering,
    AlbertTokenizerFast,
    AlbertForSequenceClassification,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator
)



In [ ]:
def load_coqa_dataset():
    return load_dataset("stanfordnlp/coqa")

dataset = load_coqa_dataset()
print(f"{len(dataset['train'])} training examples, {len(dataset['validation'])} validation examples")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/793k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7199 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

7199 training examples, 500 validation examples


In [ ]:
def prepare_coqa_features(dataset_split):
    # Lists to collect individual examples
    questions = []
    contexts = []
    answers_texts = []

    for i in range(len(dataset_split)):
        story = dataset_split[i]["story"]

        # Each example has multiple questions and answers
        for q_idx, question in enumerate(dataset_split[i]["questions"]):
            # Make sure we have a corresponding answer
            if q_idx < len(dataset_split[i]["answers"]["input_text"]):
                questions.append(question)
                contexts.append(story)
                answers_texts.append(dataset_split[i]["answers"]["input_text"][q_idx])

    # Create a list of examples with the right structure
    processed_examples = []
    for q, c, a in zip(questions, contexts, answers_texts):
        processed_examples.append({
            "question": q,
            "story": c,
            "answer": {"input_text": a}
        })

    # Apply tokenization to each example
    # tokenized_examples = [prepare_train_features(ex) for ex in processed_examples]

    return processed_examples

In [ ]:
model  = AutoModelForQuestionAnswering.from_pretrained("albert-base-v2")

tokenizer = AlbertTokenizerFast.from_pretrained("albert-base-v2", use_fast=True)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [ ]:
train_prepared = prepare_coqa_features(dataset["train"])
val_prepared = prepare_coqa_features(dataset["validation"])

In [ ]:
def tokenize_coqa_examples(examples, tokenizer=None, max_length=512):

    tokenized_examples = []

    for example in examples:
        question = example["question"]
        context = example["story"]
        answer_text = example["answer"]["input_text"]

        # Tokenize inputs
        inputs = tokenizer(
            question,
            context,
            max_length=max_length,
            truncation=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        # Convert tensors to lists to save memory
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        offset_mapping = inputs["offset_mapping"]

        # Find answer span in context
        answer_start = context.find(answer_text)
        if answer_start == -1:
            # If answer is not found, assign dummy values (typically ignored during training)
            start_positions = 0
            end_positions = 0
        else:
            answer_end = answer_start + len(answer_text)

            # Find token indices for answer
            start_token = None
            end_token = None

            for j, (start, end) in enumerate(offset_mapping):
                if start == 0 and end == 0:
                    # Skip special tokens
                    continue

                # Check if this token overlaps with answer
                if start_token is None and end > answer_start:
                    start_token = j

                if end_token is None and start >= answer_end:
                    end_token = j - 1
                    break

            start_positions = start_token if start_token is not None else 0
            end_positions = end_token if end_token is not None else len(offset_mapping) - 1

        # Add processed example to results (convert tensors to lists)
        tokenized_example = {
            "input_ids": input_ids,  # Already a list
            "attention_mask": attention_mask,  # Already a list
            "start_positions": start_positions,  # Single integer now
            "end_positions": end_positions  # Single integer now
        }

        tokenized_examples.append(tokenized_example)

    return tokenized_examples


In [ ]:
tokenized_data = tokenize_coqa_examples(train_prepared, tokenizer=tokenizer)
tokenized_data_val = tokenize_coqa_examples(val_prepared, tokenizer=tokenizer)


In [ ]:
from datasets import Dataset

# Convert list of dictionaries into a Hugging Face Dataset
tokenized_dataset = Dataset.from_list(tokenized_data)
tokenized_dataset_val = Dataset.from_list(tokenized_data_val)

# Ensure that PyTorch tensors are used during training
tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "start_positions", "end_positions"]
)
tokenized_dataset_val.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "start_positions", "end_positions"]
)

In [ ]:
# from transformers import DataCollatorWithPadding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
train_subset = tokenized_dataset.select(range(10000))
eval_subset = tokenized_dataset_val.select(range(800))

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_logits, end_logits = predictions  # Shape: (batch_size, seq_length)
    start_labels, end_labels = labels  # Shape: (batch_size,)

    # Get token index with highest probability
    start_preds = np.argmax(start_logits, axis=1)  # Shape: (batch_size,)
    end_preds = np.argmax(end_logits, axis=1)  # Shape: (batch_size,)

    # Compute accuracy
    start_accuracy = np.mean(start_preds == start_labels)
    end_accuracy = np.mean(end_preds == end_labels)

    return {"start_accuracy": start_accuracy, "end_accuracy": end_accuracy}


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )

# Train
trainer.train()

<ipython-input-14-6d3d783731fd>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Start Accuracy,End Accuracy
1,2.662000,2.096903,0.466250,0.487500
2,1.838000,2.022589,0.496250,0.507500
3,1.502700,2.196460,0.476250,0.490000


TrainOutput(global_step=1875, training_loss=1.8564547688802084, metrics={'train_runtime': 3153.9361, 'train_samples_per_second': 9.512, 'train_steps_per_second': 0.594, 'total_flos': 662514094080000.0, 'train_loss': 1.8564547688802084, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./drive/MyDrive/w266/model_checkpoints/coqa_albert_base', from_pt=True)
tokenizer.save_pretrained('./drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base', from_pt=True)

('./drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base/tokenizer_config.json',
 './drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base/special_tokens_map.json',
 './drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base/spiece.model',
 './drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base/added_tokens.json',
 './drive/MyDrive/w266/project/model_checkpoints/coqa_albert_base/tokenizer.json')